In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv("../.env")
client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [9]:
# データ読み込み
def load_and_process_data(file_path):
    df = pd.read_excel(file_path, sheet_name='売上データ')

    df["売上"] = df["単価"] * df["数量"]

    df["売上日"] = pd.to_datetime(df["売上日"])

    df["年月"] = df["売上日"].dt.strftime('%Y-%m')

    return df

In [11]:
# 集計
def sum_sales_data(df):
    pivot_table = pd.pivot_table(
        df,
        index='カテゴリー',
        columns='年月',
        values='売上',
        aggfunc='sum',
        fill_value=0
    )

    return pivot_table

In [12]:
# プロンプト作成
def convert_to_prompt(df, pivot_table):
    sales_data_text = df.astype(str)
    pivot_table_text = pivot_table.astype(str)

    prompt_text = f"""
    売上データ:
    {sales_data_text}
    月ごとのカテゴリー別売上合計：
    {pivot_table_text}
    上記の「売上データ」と「月ごとのカテゴリー別売上合計」をもとに、カテゴリー毎の売上戦略を考案してください。
    """

    return prompt_text

In [13]:
# API呼び出し

def get_openai_response(client, prompt_text, model_name="gpt-4o-mini"):
    role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt_text},
        ],
    )

    result = response.choices[0].message.content.strip()

    return result

In [14]:
# AIの出力を保存
def save_result_to_file(result, file_path="カテゴリー毎の売上戦略.md"):
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(result)
        

In [15]:
# メイン処理
def main():
    print("処理を開始します。")

    print("（1/5）データの読み込みと前処理")
    df = load_and_process_data('サンプルデータ.xlsx')

    print("（2/5）集計")
    pivot_table = sum_sales_data(df)

    print("（3/5）プロンプト生成")
    prompt_text = convert_to_prompt(df, pivot_table)

    print("（4/5）OpenAI APIからの応答を取得")
    result = get_openai_response(client, prompt_text)

    print("（5/5）結果をファイルに保存")
    save_result_to_file(result)

    print("分析結果を保存しました。")

In [16]:
main()

処理を開始します。
（1/5）データの読み込みと前処理
（2/5）集計
（3/5）プロンプト生成
（4/5）OpenAI APIからの応答を取得
（5/5）結果をファイルに保存
分析結果を保存しました。
